In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.ArcII.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3469.60it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2430.50it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 1692661.81it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3769.67it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:06<00:00, 2735.61it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 241025.78it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 4161.49it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f68303454e0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f67c9074048>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.ArcII.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.ArcII()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['kernel_1d_count'] = 32
model.params['kernel_1d_size'] = 3
model.params['kernel_2d_count'] = [64, 64]
model.params['kernel_2d_size'] = [(3, 3), (3, 3)]
model.params['pool_2d_size'] = [(3, 3), (3, 3)]

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ArcII(
  (embedding): Embedding(30058, 100)
  (conv1d_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 32, kernel_size=(3,), stride=(1,))
  )
  (conv1d_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 32, kernel_size=(3,), stride=(1,))
  )
  (matching): Matching()
  (conv2d): Sequential(
    (0): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (out): Linear(in_features=704, out_features=1, bias=True)
)
Trainab

In [9]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.751]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5524 - normalized_discounted_cumulative_gain@5(0.0): 0.6128 - mean_average_precision(0.0): 0.5743



[Iter-204 Loss-0.440]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.559 - normalized_discounted_cumulative_gain@5(0.0): 0.618 - mean_average_precision(0.0): 0.5768



[Iter-306 Loss-0.194]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5146 - normalized_discounted_cumulative_gain@5(0.0): 0.5757 - mean_average_precision(0.0): 0.5343



[Iter-408 Loss-0.129]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5227 - normalized_discounted_cumulative_gain@5(0.0): 0.5761 - mean_average_precision(0.0): 0.5422



[Iter-510 Loss-0.084]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5367 - normalized_discounted_cumulative_gain@5(0.0): 0.5917 - mean_average_precision(0.0): 0.551



[Iter-612 Loss-0.057]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5141 - normalized_discounted_cumulative_gain@5(0.0): 0.5777 - mean_average_precision(0.0): 0.5412



[Iter-714 Loss-0.044]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5504 - normalized_discounted_cumulative_gain@5(0.0): 0.5995 - mean_average_precision(0.0): 0.568



[Iter-816 Loss-0.021]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5226 - normalized_discounted_cumulative_gain@5(0.0): 0.5812 - mean_average_precision(0.0): 0.5388



[Iter-918 Loss-0.022]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.546 - normalized_discounted_cumulative_gain@5(0.0): 0.6029 - mean_average_precision(0.0): 0.5704



[Iter-1020 Loss-0.024]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5275 - normalized_discounted_cumulative_gain@5(0.0): 0.5845 - mean_average_precision(0.0): 0.5498

Cost time: 2342.0860588550568s
